In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

from qiskit.synthesis import SuzukiTrotter
from qiskit.quantum_info import Statevector
from qiskit.circuit import QuantumCircuit
from qiskit.circuit.library import PauliEvolutionGate
from qiskit import transpile

from graph_cutter import *
from utils import *

from azure.quantum import Workspace
from azure.quantum.target.microsoft import MicrosoftEstimator

workspace = Workspace (
   resource_id = "/subscriptions/e0d51919-2dda-4b7e-a8e7-77cc18580acb/resourceGroups/AzureQuantum/providers/Microsoft.Quantum/Workspaces/QECBenchmarking",
   location = "uksouth"
)

estimator = MicrosoftEstimator(workspace)

params = estimator.make_params(num_items=1)

second_order_formula = SuzukiTrotter()
L = 3
H = get_hamiltonian(L=L, J=0.2, h=1.0, alpha=np.pi / 8)
final_time = np.pi
num_timesteps = 10
dt = final_time / num_timesteps
initial_state = Statevector.from_label("000")

trotter_step_second_order = PauliEvolutionGate(H, dt, synthesis=second_order_formula)
circuit = QuantumCircuit(H.num_qubits)
for i in range(1):
    circuit.append(trotter_step_second_order, range(H.num_qubits))
target_basis = ['rx', 'ry', 'rz', 'h', 'cx']
circuit = transpile(circuit,
                       basis_gates=target_basis, 
                       optimization_level=1) 
# plot_trotter_info(circuit)
# circuit.draw('mpl', fold=-1)

In [2]:
# Error budget
params.error_budget = 0.01
params.constraints.max_t_factories = 1
job = estimator.submit(circuit, input_params=params)
r = job.get_results()
Q = r['physicalCounts']['physicalQubits']
T = r['physicalCounts']['runtime']*1e-3
print('\n', 'total physical qubits required: ', Q, '\n', 'total runtime', T)

.....6824 5680.400000000001


In [4]:
for _ in range(5):
    subcircuits, g, w = find_cuts(circuit, max_cut=8, plot=False)
    q = 0
    t = 0
    print('Sampling Overhead: ', 16**w*9**g)
    for s in subcircuits:
        if len(s.data) == 0:
            continue
        job = estimator.submit(s, input_params=params)
        r = job.get_results()
        q = max(r['physicalCounts']['physicalQubits'], q)
        t += r['physicalCounts']['runtime']*1e-3
    print('\n', 'total physical qubits required: ', q, '\n', 'total runtime', t)


Sampling Overhead:  84934656
......................
 total physical qubits required:  8020 
 total runtime 4786.0
Sampling Overhead:  150994944
.......................
 total physical qubits required:  8020 
 total runtime 4682.400000000001
Sampling Overhead:  150994944
............

RuntimeError: Cannot retrieve results as job execution failed (InvalidInputError: Algorithm requires at least one T state or measurement to estimate resources)